In [ ]:
from utils import experiment
import os

os.environ["OPENAI_API_KEY"] = "api key"

experiment.experiment(experiment_name="test_experiment", version="1.0")

실험 설정 파일을 불러옵니다: {'version': '1.0', 'test_name': 'test_experiment', 'embedding_model': 'text-embedding-3-small', 'llm_model': 'gpt-4.1-mini', 'chunk_size': 100, 'chunk_overlap': 10, 'top_k': 1}
청킹 문서를 로드 완료하였습니다.
청킹된 문서의 개수: 96397
임베딩 모델을 설정하였습니다: text-embedding-3-small
FAISS 인덱스를 불러왔습니다
retriever를 생성하였습니다. Top K: 1
RAG Chain을 생성하였습니다. LLM 모델: gpt-4.1-mini
질의 테스트를 시작합니다
Q. 버스에 관련된 내용 보여줘 

A. 문서에 따르면 버스와 관련된 내용은 다음과 같습니다.

- 기존 버스정보수집 및 버스정보제공을 위한 통신방식 현황  
- 버스정류장 및 BIT(버스정보단말기) 정보표출 유형  
- 버스 운행 관련 기초데이터 분석 (버스노선, 버스정류소, 쉘터유형 등)

이 내용들은 버스 운행과 관련된 데이터 수집, 정보 제공 방식, 그리고 버스정류장 및 노선 등의 기초 정보를 포함하고 있습니다.

질의 테스트를 종료합니다.


In [ ]:
# 리트리버 평가
from utils import recall_ev

# 파일 경로 지정
retrieved_path = "experiment/test_experiment/test_experiment.jsonl"
gold_path = "data/qa_span/span_list.json"

# 데이터 불러오기
retrieved_dict = recall_ev.load_jsonl(retrieved_path)
gold_dict = recall_ev.load_gold(gold_path)

# 점수 계산 및 출력
recall_ev.evaluate_recall_at_k(retrieved_dict, gold_dict, k=3)
recall_ev.evaluate_ndcg_at_k(retrieved_dict, gold_dict, k=3)